# **SDXL Model Converter Colab Edition**

**Be aware**

Gemini made most of the refactored code in this, and i'm still working on cleaning some content up. It MAY or MAY NOT work, but I don't have the time to fullly test it asap. Please make an note of it in the repository if there is an issue or feature request: https://github.com/Ktiseos-Nyx/SDXL_COLAB_DIFFUSERS_CONVERT

---
## **⚠️IMPORTANT: Google Colab AUP Warning & Xformers Disclaimer⚠️**
---
 **⚠️ WARNING:**
This Google Colab notebook includes tools from the `kohya-ss/sd-scripts` repository, which is **primarily designed for training Stable Diffusion models**. Using this code may cause your Google Colab account to be flagged for violating the Google Colab Acceptable Use Policy (AUP), **even if you are not actively training a model**.

**⚠️ IMPORTANT AUP RISK:**
 Using this notebook carries a **significant risk of AUP violations** which may lead to the suspension of your Google Colab account and it is provided **AS-IS**

 **⚠️ XFORMERS WARNING:**
 The `xformers` library may not be compatible with all hardware configurations or versions of PyTorch, and it may lead to errors. The use of `xformers` and the reliability of this code is entirely at your own risk.

 **⚠️AUP RISK⚠️**
 While it's not explicitly stated within the AUP, nor terms of service, slowdowns and over use on the free plans could lead to account strikes or removal. Please be aware that Ktiseos Nyx, Duskfallcrew, and even Google are not at fault for what pranks you try and pull with the AUP.

 Be patient. Be kind. Rewind.

 I'm working on a re-do of the gradio!


---


## Links

| Link Name| Description | Link |
| --- | --- | --- |
| Huggingface Backup| backup checkpoints! | [![](https://img.shields.io/static/v1?message=Open%20in%20Colab&logo=googlecolab&labelColor=5c5c5c&color=0f80c1&label=%20&style=flat)](https://github.com/Ktiseos-Nyx/HuggingFace_Backup)
|Discord| E&D Discord |[Invite](https://discord.gg/5t2kYxt7An)
|Huggingface| E&D Huggingface |[Earth & Dusk](https://huggingface.co/EarthnDusk)
|Ko-Fi| Kofi Support |[![ko-fi](https://img.shields.io/badge/Support%20me%20on%20Ko--fi-F16061?logo=ko-fi&logoColor=white&style=flat)](https://ko-fi.com/Z8Z8L4EO)
|Github| Ktiseos Nyx |[Ktiseos Nyx](https://github.com/Ktiseos-Nyx/)




## **Important Note:**

Before diving in, ensure you create a Hugging Face token with write permissions. Follow this link for instructions on token creation.

You need to create a huggingface token, go to [this link](https://huggingface.co/settings/tokens), then `create new token` or copy available token with the `Write` role.


### **Setup Instructions:**

1.  **Google Colab Environment:** This notebook is designed to run on Google Colab with a GPU runtime enabled.
2.  **Root Directory:** The `root_dir` variable specifies where files will be stored. The default is `/content`. You can specify a different location by changing the value in the corresponding text field.
3. **Repository URL:** If you want to use a specific version of the `kohya-ss/sd-scripts` repository, then you can specify the URL using the text field for "Repo URL".
4. **Branch Name:** If you want to use a specific branch of the `kohya-ss/sd-scripts` repository, then you can specify the branch name using the text field for "Branch".
5. **Xformers URL:** You are using the official xformers library, however, if you have a custom built `xformers` wheel file, then you can specify the URL using the "Xformers URL" field. (Not Recommended).
6.  **Run Setup:** After providing the values, click the "Setup Environment" button to install all required dependencies and clone the necessary files, and prepare the environment variables.
7.  **Follow the Steps:** You are now able to continue with the following steps.




In [ ]:
# @title ## **Install Kohya Script**
import os
from subprocess import getoutput
from ipywidgets import Text, Output, Button
from IPython.display import display, Markdown

# Widget creation (for the UI)
root_dir_widget = Text(value="/content", placeholder="Root Directory", description="Root Directory:")
repo_url_widget = Text(value="https://github.com/kohya-ss/sd-scripts", placeholder="Repo URL", description="Repo URL:")
branch_widget = Text(value="sdxl", placeholder="Branch", description="Branch:")
output_widget = Output()
setup_button = Button(description="Setup Environment")


def setup_environment(root_dir, repo_url, branch, output_widget):
    """Sets up the environment: creates directories, clones repo, installs dependencies."""
    try:
        repo_dir, models_dir, tools_dir, vae_dir = create_directories(root_dir)
        clone_repo(repo_url, repo_dir, branch, output_widget)
        install_dependencies(output_widget)
        prepare_environment()

        with output_widget:
            print(f"Root directory set to: {root_dir}")
            print("Setup Complete")

    except Exception as e:
        with output_widget:
            print(f"An error occurred: {e}")

def create_directories(root_dir):
    """Creates the necessary directories."""
    repo_dir = os.path.join(root_dir, "kohya-trainer")
    models_dir = os.path.join(root_dir, "models")
    tools_dir = os.path.join(repo_dir, "tools")
    vae_dir = os.path.join(root_dir, "vae")

    os.makedirs(models_dir, exist_ok=True)
    os.makedirs(vae_dir, exist_ok=True)
    os.makedirs(tools_dir, exist_ok=True)
    os.makedirs(repo_dir, exist_ok=True)

    return repo_dir, models_dir, tools_dir, vae_dir

def clone_repo(url, dir, branch, output_widget):
    """Clones the specified repository."""
    try:
        if not os.path.exists(dir):
            !git clone -b {branch} {url} {dir}
    except Exception as e:
        with output_widget:
            print(f"Error cloning repo: {e}")

def install_dependencies(output_widget):
    """Installs the required dependencies."""
    try:
        !apt install aria2
        !pip install -q torch==2.1.0+cu121 diffusers[torch]==0.25.0 transformers==4.36.0 einops==0.6.0 open-clip-torch==2.20.0 invisible-watermark  jax[cuda12_pip]==0.4.23  -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
        !pip install -q xformers==0.0.23
    except Exception as e:
        with output_widget:
            print(f"Error installing dependencies: {e}")

def prepare_environment():
    """Sets environment variables."""
    os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
    os.environ["SAFETENSORS_FAST_GPU"] = "1"
    os.environ["PYTHONWARNINGS"] = "ignore"
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "garbage_collection_threshold:0.9,max_split_size_mb:512"
    os.environ["TCMALLOC_AGGRESSIVE_DECOMMIT"] = "t"
    os.environ["CUDA_MODULE_LOADING"] = "LAZY"


def main_setup(b):
    """Main function for setting up the environment (linked to the setup button)."""
    root_dir = root_dir_widget.value
    repo_url = repo_url_widget.value
    branch = branch_widget.value

    setup_environment(root_dir, repo_url, branch, output_widget)


# Link the button to the main setup function
setup_button.on_click(main_setup)

# Display the input widgets and setup button
display(root_dir_widget)
display(repo_url_widget)
display(branch_widget)
display(setup_button)
display(output_widget)








In [ ]:
# @title ### ♻ **Clean Folder**
from IPython.display import display, Markdown
import os
import shutil

# Function to clear and delete a folder
def clear_and_delete_folder(colab_folder_path):
    try:
        # Use shutil.rmtree to remove all files and subdirectories
        shutil.rmtree(colab_folder_path)
        display(Markdown(f"Deleted all contents in folder: `{colab_folder_path}`"))
    except Exception as e:
        display(Markdown(f"Error deleting folder `{colab_folder_path}`: {e}"))

# @markdown ### Folder Path for Deletion
colab_folder_path = "" # @param {type: "string"}

# Call the function to clear and delete the folder
clear_and_delete_folder(colab_folder_path)

In [ ]:
# @title ### ♻ **Fix Before Converting (Optional, may not work)**
#@markdown This MAY not work for SDXL, if you require key fixings and this works let me know!

from IPython.display import display, Markdown
from safetensors import safe_open
from safetensors.torch import save_file
import os

#@markdown ### Model Type
model_type = "" # @param ["", "vae"]

#@markdown ### Input path
load_path = "" # @param {type: "string"}

#@markdown ### Output path
save_path = "" # @param {type: "string"}


def fix_diffusers_model_conversion(load_path, save_path, model_type):
   try:

    # load original
    tensors = {}
    with safe_open(load_path, framework="pt") as f:
        for key in f.keys():
            tensors[key] = f.get_tensor(key)

        # migrate
        new_tensors = {}
        for k, v in tensors.items():
            new_key = k
            # only fix the vae
            if model_type == "vae" and 'first_stage_model.' in k:
                # migrate q, k, v keys
                new_key = new_key.replace('.to_q.weight', '.q.weight')
                new_key = new_key.replace('.to_q.bias', '.q.bias')
                new_key = new_key.replace('.to_k.weight', '.k.weight')
                new_key = new_key.replace('.to_k.bias', '.k.bias')
                new_key = new_key.replace('.to_v.weight', '.v.weight')
                new_key = new_key.replace('.to_v.bias', '.v.bias')
            new_tensors[new_key] = v

        # save
        save_file(new_tensors, save_path)
        display(Markdown(f"Keys have been fixed and the model saved to {save_path}"))

   except Exception as e:
       display(Markdown(f"An error occurred during key fixing: {e}"))

# Example usage
fix_diffusers_model_conversion(load_path, save_path, model_type)

## **Download XL MODELS**

**Instructions:**

1. You must specify a Hugging Face Token if the model you are downloading is in a private Hugging Face Repository.
2. You must specify a URL to an SDXL model, which can be a Google Drive Link, a Hugging Face Model or direct URL to a file.
3. You may optionally specify a folder where you would like to save the file. If you do not specify one, it will be stored in `/content/models` folder.
4. Click the "Download SDXL Model" to begin the process.


In [ ]:

import os
import re
import json
import glob
import gdown
import requests
import subprocess
from urllib.parse import urlparse, unquote
from pathlib import Path
from ipywidgets import Text, Button, Output
from IPython.display import display, Markdown

# @markdown Place your Huggingface [Read Token](https://huggingface.co/settings/tokens) Here.
HUGGINGFACE_TOKEN = ""#@param {type: "string"}

# @markdown Place your SDXL Model URL Here.
SDXL_MODEL_URL = "" #@param {type: "string"}

#@markdown Specify the output path for the files here
OUTPUT_PATH = "/content/models" #@param {type: "string"}

# Create a button to trigger the download
download_button = Button(description="Download SDXL Model")

# Create an output widget to display the results
output_widget = Output()

def download_sdxl_model(url, dst, huggingface_token, output_widget):
    """Downloads an SDXL model from the given URL."""
    def get_supported_extensions():
        return tuple([".ckpt", ".safetensors", ".pt", ".pth"])

    def get_filename(url):
        extensions = get_supported_extensions()

        if url.endswith(tuple(extensions)):
            filename = os.path.basename(url)
        else:
            response = requests.get(url, stream=True)
            response.raise_for_status()

            if 'content-disposition' in response.headers:
                content_disposition = response.headers['content-disposition']
                filename = re.findall('filename="?([^"]+)"?', content_disposition)[0]
            else:
                url_path = urlparse(url).path
                filename = unquote(os.path.basename(url_path))

        if filename.endswith(tuple(get_supported_extensions())):
            return filename
        else:
            return None

    def parse_args(config):
        args = []

        for k, v in config.items():
            if k.startswith("_"):
                args.append(f"{v}")
            elif isinstance(v, str) and v is not None:
                args.append(f'--{k}={v}')
            elif isinstance(v, bool) and v:
                args.append(f"--{k}")
            elif isinstance(v, float) and not isinstance(v, bool):
                args.append(f"--{k}={v}")
            elif isinstance(v, int) and not isinstance(v, bool):
                args.append(f"--{k}={v}")

        return args

    def aria2_download(dir, filename, url, huggingface_token):
        user_header = f"Authorization: Bearer {huggingface_token}"

        aria2_config = {
            "console-log-level"         : "error",
            "summary-interval"          : 10,
            "header"                    : user_header if "huggingface.co" in url else None,
            "continue"                  : True,
            "max-connection-per-server" : 16,
            "min-split-size"            : "1M",
            "split"                     : 16,
            "dir"                       : dir,
            "out"                       : filename,
            "_url"                      : url,
        }
        aria2_args = parse_args(aria2_config)
        subprocess.run(["aria2c", *aria2_args])

    def gdown_download(url, dst, filepath):
        if "/uc?id/" in url or "/file/d/" in url:
            return gdown.download(url, filepath, quiet=False)
        elif "/drive/folders/" in url:
            os.chdir(dst)
            return gdown.download_folder(url, quiet=True, use_cookies=False)

    filename = get_filename(url)
    filepath = os.path.join(dst, filename)

    try:
        if "drive.google.com" in url:
            gdown = gdown_download(url, dst, filepath)
        elif url.startswith("/content/drive/MyDrive/"):
            return url
        else:
            if "huggingface.co" in url:
                if "/blob/" in url:
                    url = url.replace("/blob/", "/resolve/")
            aria2_download(dst, filename, url, huggingface_token)
        with output_widget:
            print(f"Model downloaded to: {dst}")
    except Exception as e:
        with output_widget:
            print(f"An error occurred during the download: {e}")


def main_download(b):
    """Main download function linked to the download button."""
    try:
        download_sdxl_model(SDXL_MODEL_URL, OUTPUT_PATH, HUGGINGFACE_TOKEN, output_widget)
    except Exception as e:
        with output_widget:
            print(f"An error occurred in the main download function: {e}")


# Link the download button to the main download function
download_button.on_click(main_download)

# Display the download-related widgets
display(download_button)
display(output_widget)


In [ ]:


# @title ## **Convert SDXL to Diffusers**
import os
import urllib.request
from ipywidgets import Text, Button, Output
from IPython.display import display, Markdown

#@markdown ### **Conversion Config**
model_to_load = "" #@param {'type': 'string'}
script_url = "https://raw.githubusercontent.com/Ktiseos-Nyx/SDXL_COLAB_DIFFUSERS_CONVERT/refs/heads/main/sdxl_to_diffusers_2025_colab.py" #@param {'type': 'string'}
global_step = 0 #@param {type:"integer"}
epoch = 0 #@param {type:"integer"}
save_precision_as = "fp16" #@param ["fp16","bf16","float"] {'allow-input': false}
reference_model = "stabilityai/stable-diffusion-xl-base-1.0" #@param {'type': 'string'}
output_path = "/content/output" #@param {type: "string"}

# Create a button to trigger the conversion
convert_button = Button(description="Convert SDXL to Diffusers")
output_widget = Output()

import argparse
import os
import torch
from diffusers import StableDiffusionXLPipeline
#import library.sdxl_model_util as sdxl_model_util

def convert_model(args):
    load_dtype = torch.float16 if args.fp16 else None
    save_dtype = get_save_dtype(args)

    is_load_checkpoint = determine_load_checkpoint(args.model_to_load)
    is_save_checkpoint = not is_load_checkpoint  # reverse of load model

    loaded_model_data = load_sdxl_model(args, is_load_checkpoint, load_dtype)
    convert_and_save_sdxl_model(args, is_save_checkpoint, loaded_model_data, save_dtype)

def get_save_dtype(args):
    if args.save_precision_as == "fp16":
        return torch.float16
    elif args.save_precision_as == "bf16":
        return torch.bfloat16
    elif args.save_precision_as == "float":
        return torch.float
    else:
        return None

def determine_load_checkpoint(model_to_load):
    if model_to_load.endswith('.ckpt') or model_to_load.endswith('.safetensors'):
        return True
    elif os.path.isdir(model_to_load):
        required_folders = {"unet", "text_encoder", "text_encoder_2", "tokenizer", "tokenizer_2", "scheduler", "vae"}
        if required_folders.issubset(set(os.listdir(model_to_load))) and os.path.isfile(os.path.join(model_to_load, "model_index.json")):
            return False
    return None  # handle this case as required

def load_sdxl_model(args, is_load_checkpoint, load_dtype):
    model_load_message = "checkpoint" if is_load_checkpoint else "Diffusers" + (" as fp16" if args.fp16 else "")
    print(f"Loading {model_load_message}: {args.model_to_load}")

    if is_load_checkpoint:
        loaded_model_data = load_from_sdxl_checkpoint(args)
    else:
        loaded_model_data = load_sdxl_from_diffusers(args, load_dtype)

    return loaded_model_data

def load_from_sdxl_checkpoint(args):
    #text_encoder1, text_encoder2, vae, unet, _, _ = sdxl_model_util.load_models_from_sdxl_checkpoint(
    #    "sdxl_base_v1-0", args.model_to_load, "cpu"
    #)

    #Implement Load model from ckpt or safetensors

    #Define these at the end.
    text_encoder1, text_encoder2, vae, unet = None,None,None,None
    print("Loading from Checkpoint not implemented, please implement based on your model needs")
    return text_encoder1, text_encoder2, vae, unet

def load_sdxl_from_diffusers(args, load_dtype):
    pipeline = StableDiffusionXLPipeline.from_pretrained(
        args.model_to_load, torch_dtype=load_dtype, tokenizer=None, tokenizer_2=None, scheduler=None
        )
    text_encoder1 = pipeline.text_encoder
    text_encoder2 = pipeline.text_encoder_2
    vae = pipeline.vae
    unet = pipeline.unet

    return text_encoder1, text_encoder2, vae, unet

def convert_and_save_sdxl_model(args, is_save_checkpoint, loaded_model_data, save_dtype):
    text_encoder1, text_encoder2, vae, unet = loaded_model_data
    model_save_message = "checkpoint" + ("" if save_dtype is None else f" in {save_dtype}") if is_save_checkpoint else "Diffusers"
    print(f"Converting and saving as {model_save_message}: {args.model_to_save}")

    if is_save_checkpoint:
        save_sdxl_as_checkpoint(args, text_encoder1, text_encoder2, vae, unet, save_dtype)
    else:
        save_sdxl_as_diffusers(args,  text_encoder1, text_encoder2, vae, unet, save_dtype)

def save_sdxl_as_checkpoint(args, text_encoder1, text_encoder2, vae, unet, save_dtype):
    logit_scale = None
    ckpt_info = None

    #key_count = sdxl_model_util.save_stable_diffusion_checkpoint(
    #    args.model_to_save, text_encoder1, text_encoder2, unet, args.epoch, args.global_step, ckpt_info, vae, logit_scale, save_dtype
    #    )

    print(f"Saving as Checkpoint not implemented, please implement based on your model needs")
    #print(f"Model saved. Total converted state_dict keys: {key_count}")

def save_sdxl_as_diffusers(args, text_encoder1, text_encoder2, vae, unet, save_dtype):
    reference_model_message = args.reference_model if args.reference_model is not None else 'default model'
    print(f"Copying scheduler/tokenizer config from: {reference_model_message}")
    #sdxl_model_util.save_diffusers_checkpoint(
    #    args.model_to_save, text_encoder1, text_encoder2, unet, args.reference_model, vae, True, save_dtype
    #)

    # Save diffusers pipeline
    pipeline = StableDiffusionXLPipeline(
        vae=vae,
        text_encoder=text_encoder1,
        text_encoder_2=text_encoder2,
        unet=unet,
        scheduler=None, # Replace None if there is a scheduler
        tokenizer=None, # Replace None if there is a tokenizer
        tokenizer_2=None # Replace None if there is a tokenizer_2
    )

    pipeline.save_pretrained(args.model_to_save)
    print(f"Model saved as {save_dtype}.")

def get_save_path(args, is_save_checkpoint):
    basename = os.path.splitext(args.model_to_load)[0]
    if is_save_checkpoint:
        return increment_filename(basename)
    else:
        return increment_filename(basename + ".safetensors")

def increment_filename(filename):
    base, ext = os.path.splitext(filename)
    counter = 1
    while os.path.exists(filename):
        filename = f"{base}({counter}){ext}"
        counter += 1
    return filename

def setup_parser() -> argparse.ArgumentParser:
    parser = argparse.ArgumentParser()
    parser.add_argument("--fp16", action="store_true", help="load as fp16 (Diffusers only)")
    parser.add_argument("--save_precision_as", type=str, default="no", choices=["fp16", "bf16", "float"], help="save precision")
    parser.add_argument("--epoch", type=int, default=0, help="epoch to write to checkpoint")
    parser.add_argument("--global_step", type=int, default=0, help="global_step to write to checkpoint")
    parser.add_argument("--reference_model", type=str, default=None, help="reference Diffusers model to copy scheduler/tokenizer config from, used when saving as Diffusers format, default is `runwayml/stable-diffusion-v1-5` or `stabilityai/stable-diffusion-2-1`")
    parser.add_argument("--model_to_load", type=str, required=True, help="model to load: checkpoint file or Diffusers model's directory")
    return parser

def main_convert(b):
    #Manually configure the args to feed into convert model
    class Args:
        def __init__(self, model_to_load, save_precision_as, epoch, global_step, reference_model, output_path, fp16):
            self.model_to_load = model_to_load
            self.save_precision_as = save_precision_as
            self.epoch = epoch
            self.global_step = global_step
            self.reference_model = reference_model
            self.output_path = output_path
            self.fp16 = fp16

    args = Args(model_to_load, save_precision_as, epoch, global_step, reference_model, output_path, False)
    args.model_to_save = get_save_path(args, determine_load_checkpoint(args.model_to_load))

    convert_model(args)
    with output_widget:
        print(f"Conversion complete to {output_path}!")


# Link the button to the main function
convert_button.on_click(main_convert)

# Display the widgets
display(convert_button)
display(output_widget)


## **Inference**


### **Run pipeline**

In [ ]:
import torch
from torch import autocast
from diffusers.models import AutoencoderKL
from diffusers import StableDiffusionXLPipeline, EulerAncestralDiscreteScheduler

model = "/content/models/counterfeitxl_"
vae = AutoencoderKL.from_pretrained("stabilityai/sdxl-vae")

pipe = StableDiffusionXLPipeline.from_pretrained(
    model,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
    vae=vae
    )
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
pipe.to('cuda')

### **Generate**

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

prompt = "masterpiece, best quality, 1girl, green hair, sweater, looking at viewer, upper body, beanie, outdoors, watercolor, night, turtleneck"
negative_prompt = "lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry"
output = "/content/anime_girl.png"

image = pipe(
    prompt,
    negative_prompt=negative_prompt,
    width=1024,
    height=1024,
    guidance_scale=12,
    target_size=(1024,1024),
    original_size=(4096,4096),
    num_inference_steps=50
    ).images[0]

image.save(output)
image = Image.open(output)
plt.imshow(image)
plt.axis('off') # to hide the axis
plt.show()


### **Clear Memory**

In [ ]:
import gc

if "pipe" in globals():
    del pipe
if "image" in globals():
    del image
torch.cuda.empty_cache()
gc.collect()

## Deploy

In [ ]:
# @title ### **Huggingface Hub config**
from huggingface_hub import login, HfApi
from huggingface_hub.utils import validate_repo_id, HfHubHTTPError

# @markdown Login to Huggingface Hub
# @markdown > Get **your** huggingface `WRITE` token [here](https://huggingface.co/settings/tokens)
write_token = ""  # @param {type:"string"}
# @markdown Fill this if you want to upload to your organization, or just leave it empty.
orgs_name = ""  # @param{type:"string"}
# @markdown If your model repo does not exist, it will automatically create it.
model_name = ""  # @param {type:"string"}
make_private = False  # @param{type:"boolean"}

def authenticate(write_token):
    login(write_token, add_to_git_credential=True)
    api = HfApi()
    return api.whoami(write_token), api

def create_model_repo(api, user, orgs_name, model_name, make_private=False):
    if orgs_name == "":
        repo_id = user["name"] + "/" + model_name.strip()
    else:
        repo_id = orgs_name + "/" + model_name.strip()

    try:
        validate_repo_id(repo_id)
        api.create_repo(repo_id=repo_id, repo_type="model", private=make_private)
        print(f"Model repo '{repo_id}' didn't exist, creating repo")
    except HfHubHTTPError as e:
        print(f"Model repo '{repo_id}' exists, skipping create repo")

    print(f"Model repo '{repo_id}' link: https://huggingface.co/{repo_id}\n")

    return repo_id

user, api = authenticate(write_token)

if model_name:
    model_repo = create_model_repo(api, user, orgs_name, model_name, make_private)


In [ ]:
# @title ### **Upload Model**
# @title ### **Upload to Huggingface**
from huggingface_hub import HfApi
from pathlib import Path
import os

api = HfApi()

# @markdown This will be uploaded to model repo
model_path = "/content/models/counterfeitxl_"  # @param {type :"string"}
path_in_repo = ""  # @param {type :"string"}
project_name = os.path.basename(model_path)

# @markdown Other Information
commit_message = ""  # @param {type :"string"}

def is_diffusers_model(model_path):
    required_folders = {"unet", "text_encoder", "text_encoder_2", "tokenizer", "tokenizer_2", "scheduler", "vae"}
    return required_folders.issubset(set(os.listdir(model_path))) and os.path.isfile(os.path.join(model_path, "model_index.json"))

def upload_model(model_paths, is_folder: bool):
    path_obj = Path(model_paths)
    trained_model = path_obj.parts[-1]

    path_in_repo_local = path_in_repo if path_in_repo and not is_diffusers_model(model_paths) else ""

    notification = f"Uploading {trained_model} from {model_paths} to https://huggingface.co/{model_repo}"
    print(notification)

    if is_folder:
        if is_diffusers_model(model_paths):
            commit_message = f"Upload diffusers format: {trained_model}"
            print("Detected diffusers model. Adjusting upload parameters.")
        else:
            commit_message = f"Upload checkpoint: {trained_model}"
            print("Detected regular model. Adjusting upload parameters.")

        api.upload_folder(
            folder_path=model_paths,
            path_in_repo=path_in_repo_local,
            repo_id=model_repo,
            commit_message=commit_message,
            ignore_patterns=".ipynb_checkpoints",
        )
    else:
        commit_message = f"Upload file: {trained_model}"
        api.upload_file(
            path_or_fileobj=model_paths,
            path_in_repo=path_in_repo_local,
            repo_id=model_repo,
            commit_message=commit_message,
        )

    success_notification = f"Upload successful. Check the model at https://huggingface.co/{model_repo}/tree/main"
    print(success_notification)

def upload():
    if model_path.endswith((".ckpt", ".safetensors", ".pt")):
        upload_model(model_path, False)
    else:
        upload_model(model_path, True)

upload()



>## Credits:


| Patched Origin | Description | Link |
| --- | --- | --- |
|Patched from| ARCHIVED |[SDXL - Linaqruf](https://colab.research.google.com/github/Linaqruf/sdxl-model-converter/blob/main/sdxl_model_converter.ipynb)
|***Linaqruf @ Github***: |https://github.com/Linaqruf
|Linaqruf Ko-Fi | [![](https://dcbadge.vercel.app/api/shield/850007095775723532?style=flat)](https://lookup.guru/850007095775723532) [![ko-fi](https://img.shields.io/badge/Support%20me%20on%20Ko--fi-F16061?logo=ko-fi&logoColor=white&style=flat)](https://ko-fi.com/linaqruf)
| Linaqruf Saweria |<a href="https://saweria.co/linaqruf"><img alt="Saweria" src="https://img.shields.io/badge/Saweria-7B3F00?style=flat&logo=ko-fi&logoColor=white"/></a>

